## Training 

The main task on this dataset is to predict based on the given attributes of a patient that whether that particular person has heart disease or not and other is the experimental task to diagnose and find out various insights from this dataset which could help in understanding the problem more.

#### Libraries and modules to use

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import mlflow

#### Data reading

In [1]:
import pandas as pd

file_url = "https://mlrawdata123.blob.core.windows.net/rawdata/raw_data.csv"
df = pd.read_csv(file_url)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [23]:
df.loc[0].to_json()

'{"age":63,"sex":1,"cp":1,"trestbps":145,"chol":233,"fbs":1,"restecg":2,"thalach":150,"exang":0,"oldpeak":2.3,"slope":3,"ca":0,"thal":"fixed","target":0}'

In [6]:
from app.services.model import ModelService
from app.services.azureconn import AzuremlCon


azmlcon = AzuremlCon()
ms = ModelService(azmlcon, model_name='heart_disease_model', model_version="Production")

c:\Users\ASUS\Anaconda3\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


2024/01/02 13:31:46 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.9.2, required: mlflow==2.4)
 - cloudpickle (current: 2.0.0, required: cloudpickle==2.2.1)
 - numpy (current: 1.21.5, required: numpy==1.25.0)
 - packaging (current: 21.3, required: packaging==23.0)
 - scikit-learn (current: 1.2.0, required: scikit-learn==1.2.2)
 - scipy (current: 1.9.1, required: scipy==1.10.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/01/02 13:31:46 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.11`, differs from the version of Python that is currently running, `Python 3.9.13`, and may be incompatible
c:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator OneHotEncoder from version 

In [8]:
from app.schemas.raw_data import raw_data
import pandas as pd

pd.DataFrame.from_records([raw_data.model_json_schema()["examples"][0]])

,age,ca,chol,cp,exang,fbs,oldpeak,restecg,sex,slope,thal,thalach,trestbps
0,63,0,233,1,0,1,2.3,2,1,3,fixed,150,145


In [9]:
data = raw_data.model_json_schema()["examples"][0]
data2 = raw_data(**data)
data2.model_dump()

{'age': 63,
 'sex': 1,
 'cp': 1,
 'trestbps': 145,
 'chol': 233,
 'fbs': 1,
 'restecg': 2,
 'thalach': 150,
 'exang': 0,
 'oldpeak': 2.3,
 'slope': 3,
 'ca': 0,
 'thal': 'fixed'}

In [12]:
model = azmlcon.load_model(model_name='heart_disease_model', model_version="Production")

2024/01/02 13:33:45 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.9.2, required: mlflow==2.4)
 - cloudpickle (current: 2.0.0, required: cloudpickle==2.2.1)
 - numpy (current: 1.21.5, required: numpy==1.25.0)
 - packaging (current: 21.3, required: packaging==23.0)
 - scikit-learn (current: 1.2.0, required: scikit-learn==1.2.2)
 - scipy (current: 1.9.1, required: scipy==1.10.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/01/02 13:33:45 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.11`, differs from the version of Python that is currently running, `Python 3.9.13`, and may be incompatible
c:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator OneHotEncoder from version 

In [15]:
model.unwrap_python_model()

MlflowException: Unable to retrieve base model object from pyfunc.

In [5]:
ms.row_prediction(data2.model)

ValueError: Expected 2D array, got 1D array instead:
array=[{'age': 63, 'sex': 1, 'cp': 1, 'trestbps': 145, 'chol': 233, 'fbs': 1, 'restecg': 2, 'thalach': 150, 'exang': 0, 'oldpeak': 2.3, 'slope': 3, 'ca': 0, 'thal': 'fixed'}].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [15]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

#### Data preprocessing

In [3]:
df["thal"] = df["thal"].astype("category").cat.codes

Train-test split

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

Define numerical and categorical features

In [5]:
num_features = X_train.select_dtypes('number').columns.tolist()
cat_features = X_train.select_dtypes('object').columns.tolist()

Define preprocessing pipeline

In [6]:
column_transformer = ColumnTransformer(
    [("OHE", OneHotEncoder(sparse_output=False), cat_features),
     ("scaler", MinMaxScaler(), num_features)
     ]
).set_output(transform="pandas")

### Modeling

Set experiment

In [7]:
mlflow.delete_experiment("heart_disease")

RestException: BAD_REQUEST: Response: {'Error': {'Code': 'UserError', 'Severity': None, 'Message': 'Experiment ID must be a GUID - cannot delete based on experiment name.', 'MessageFormat': None, 'MessageParameters': None, 'ReferenceCode': None, 'DetailsUri': None, 'Target': None, 'Details': [], 'InnerError': None, 'DebugInfo': None, 'AdditionalInfo': None}, 'Correlation': {'operation': 'ba198ff3dcd5de2c31be144b5c76348f', 'request': '03dbe9f8441eb1db'}, 'Environment': 'centralus', 'Location': 'centralus', 'Time': '2023-12-19T03:49:54.6256373+00:00', 'ComponentName': 'mlflow', 'statusCode': 400, 'error_code': 'BAD_REQUEST'}

In [ ]:
mlflow.search_experiments()

[]

In [8]:
mlflow.set_experiment(experiment_name="heart_disease_exp")

2023/12/19 03:50:04 INFO mlflow.tracking.fluent: Experiment with name 'heart_disease_exp' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1702957804159, experiment_id='d8fd18d1-884d-4a8d-8f4a-8eddef71c4d2', last_update_time=None, lifecycle_stage='active', name='heart_disease_exp', tags={}>

#### Logistic regression

In [9]:
with mlflow.start_run(run_name = 'log_reg_baseline'):
    model_name = LogisticRegression().__class__.__name__
    mlflow.set_tag('model_name',model_name)

    pipe = Pipeline(steps = [
        ("transformers", column_transformer),
        (model_name, LogisticRegression(random_state=1234, penalty='l2'))
    ])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    y_pred_prob = pipe.predict_proba(X_test)[:,1]

    accuracy = metrics.accuracy_score(y_true=y_test, y_pred=y_pred)
    recall = metrics.recall_score(y_true=y_test, y_pred=y_pred)
    precision = metrics.recall_score(y_true=y_test, y_pred=y_pred)
    auc_score = metrics.roc_auc_score(y_true=y_test, y_score=y_pred_prob)

    mlflow.log_metrics({"test_accuracy": accuracy, "test_recall": recall, "test_precision": precision,
                        "test_auc": auc_score})
    
    mlflow.log_param("penalty", "l2")

    mlflow.sklearn.log_model(pipe, f"heart_disease_{model_name}")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


#### Decision Tree

In [11]:
with mlflow.start_run(run_name = 'decision_tree'):
    model_name = DecisionTreeClassifier().__class__.__name__
    mlflow.set_tag('model_name',model_name)

    pipe = Pipeline(steps = [
        ("transformers", column_transformer),
        (model_name, DecisionTreeClassifier(random_state=1234, max_depth=10))
    ])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    y_pred_prob = pipe.predict_proba(X_test)[:,1]

    accuracy = metrics.accuracy_score(y_true=y_test, y_pred=y_pred)
    recall = metrics.recall_score(y_true=y_test, y_pred=y_pred)
    precision = metrics.recall_score(y_true=y_test, y_pred=y_pred)
    auc_score = metrics.roc_auc_score(y_true=y_test, y_score=y_pred_prob)

    mlflow.log_metrics({"test_accuracy": accuracy, "test_recall": recall, "test_precision": precision,
                        "test_auc": auc_score})

    mlflow.log_param("max_depth", 10)

    mlflow.sklearn.log_model(pipe, f"heart_disease_{model_name}")

#### Random Forest

In [12]:
with mlflow.start_run(run_name = 'random_forest'):
    model_name = RandomForestClassifier().__class__.__name__
    mlflow.set_tag('model_name',model_name)

    pipe = Pipeline(steps = [
        ("transformers", column_transformer),
        (model_name, RandomForestClassifier(random_state=1234, n_estimators=50))
    ])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    y_pred_prob = pipe.predict_proba(X_test)[:,1]

    accuracy = metrics.accuracy_score(y_true=y_test, y_pred=y_pred)
    recall = metrics.recall_score(y_true=y_test, y_pred=y_pred)
    precision = metrics.recall_score(y_true=y_test, y_pred=y_pred)
    auc_score = metrics.roc_auc_score(y_true=y_test, y_score=y_pred_prob)

    mlflow.log_metrics({"test_accuracy": accuracy, "test_recall": recall, "test_precision": precision,
                        "test_auc": auc_score})

    mlflow.log_param("n_estimators", 50)

    mlflow.sklearn.log_model(pipe, f"heart_disease_{model_name}")

### Model Registry

Set MLflow client

In [31]:
from mlflow.tracking import MlflowClient

ML_FLOW_TRACKING_URI = mlflow.get_tracking_uri()
client = MlflowClient(tracking_uri=ML_FLOW_TRACKING_URI)
client.search_experiments()

[<Experiment: artifact_location='', creation_time=1702957804159, experiment_id='d8fd18d1-884d-4a8d-8f4a-8eddef71c4d2', last_update_time=None, lifecycle_stage='active', name='heart_disease_exp', tags={}>]

In [29]:
experiment_id = mlflow.get_experiment_by_name("heart_disease_exp").experiment_id

Search runs with accuracy higher than 0.79

In [61]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=experiment_id,
    filter_string='metrics.test_accuracy > 0.79',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=3
)

runs

[<Run: data=<RunData: metrics={'test_accuracy': 0.7912087912087912,
  'test_auc': 0.7301587301587301,
  'test_precision': 0.5714285714285714,
  'test_recall': 0.5714285714285714}, params={'max_depth': '10'}, tags={'mlflow.rootRunId': '8bb212b8-6697-4960-b9bc-0dff1bbb0d2d',
  'mlflow.runName': 'decision_tree',
  'mlflow.user': 'Jose Luis Alcocer Cáceres',
  'model_name': 'DecisionTreeClassifier'}>, info=<RunInfo: artifact_uri='', end_time=1702957858531, experiment_id='d8fd18d1-884d-4a8d-8f4a-8eddef71c4d2', lifecycle_stage='active', run_id='8bb212b8-6697-4960-b9bc-0dff1bbb0d2d', run_name='decision_tree', run_uuid='8bb212b8-6697-4960-b9bc-0dff1bbb0d2d', start_time=1702957854331, status='FINISHED', user_id='Jose Luis Alcocer Cáceres'>, inputs=<RunInputs: dataset_inputs=[]>>,
 <Run: data=<RunData: metrics={'test_accuracy': 0.8241758241758241,
  'test_auc': 0.9084467120181405,
  'test_precision': 0.5357142857142857,
  'test_recall': 0.5357142857142857}, params={'n_estimators': '50'}, tags={'

In [66]:
for run in runs:
    artifact = client.list_artifacts(run_id=run.info.run_id)[0].path
    print(f"run id: {run.info.run_id}, test_accuracy: {run.data.metrics['test_accuracy']:.4f},  artifact: {artifact}")

run id: 8bb212b8-6697-4960-b9bc-0dff1bbb0d2d, test_accuracy: 0.7912,  artifact: heart_disease_DecisionTreeClassifier
run id: c181a6c4-3f0f-42e6-8e8f-6c6c6167194f, test_accuracy: 0.8242,  artifact: heart_disease_RandomForestClassifier


Registry Random Forest Classifier

In [67]:
run_id = "c181a6c4-3f0f-42e6-8e8f-6c6c6167194f"
artifact = "heart_disease_RandomForestClassifier"
model_uri = f'runs:/{run_id}/{artifact}'
mlflow.register_model(model_uri=model_uri, name = 'heart_disease_model')

Successfully registered model 'heart_disease_model'.
2023/12/19 04:22:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: heart_disease_model, version 1
Created version '1' of model 'heart_disease_model'.


<ModelVersion: aliases=[], creation_timestamp=1702959727174, current_stage='None', description='', last_updated_timestamp=1702959727174, name='heart_disease_model', run_id='c181a6c4-3f0f-42e6-8e8f-6c6c6167194f', run_link='', source='azureml://centralus.api.azureml.ms/mlflow/v2.0/subscriptions/fe465efa-7c36-4422-ab58-265e48560849/resourceGroups/machine-learning/providers/Microsoft.MachineLearningServices/workspaces/machine-learning-space/experiments/d8fd18d1-884d-4a8d-8f4a-8eddef71c4d2/runs/c181a6c4-3f0f-42e6-8e8f-6c6c6167194f/artifacts/heart_disease_RandomForestClassifier', status='READY', status_message='', tags={}, user_id='', version='1'>

Change stage to production

In [70]:
model_name = 'heart_disease_model'
model_version = 1
new_stage = 'Production'
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1702959727174, current_stage='Production', description='', last_updated_timestamp=1702959791455, name='heart_disease_model', run_id='c181a6c4-3f0f-42e6-8e8f-6c6c6167194f', run_link='', source='azureml://centralus.api.azureml.ms/mlflow/v2.0/subscriptions/fe465efa-7c36-4422-ab58-265e48560849/resourceGroups/machine-learning/providers/Microsoft.MachineLearningServices/workspaces/machine-learning-space/experiments/d8fd18d1-884d-4a8d-8f4a-8eddef71c4d2/runs/c181a6c4-3f0f-42e6-8e8f-6c6c6167194f/artifacts/heart_disease_RandomForestClassifier', status='READY', status_message='', tags={}, user_id='', version='1'>

In [5]:
from dotenv import load_dotenv

load_dotenv('./secrets.env')

True

In [6]:
import os
WORKSPACE = os.getenv('WORKSPACE')
WORKSPACE_LOCATION = os.getenv('WORKSPACE_LOCATION')
SUBSCRIPTION_ID = os.getenv('SUBSCRIPTION_ID')
RESOURCE_GROUP = os.getenv('RESOURCE_GROUP')

In [7]:
from mlflow.tracking import MlflowClient
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

In [14]:
import fastapi

fastapi.__version__


'0.104.0'

In [8]:
ml_client = MLClient(
    DefaultAzureCredential(), SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE
)

In [9]:
azureml_tracking_uri = ml_client.workspaces.get(
    ml_client.workspace_name
).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_tracking_uri)

In [11]:
model_name = "heart_disease_model"
model_local_path = "model"
mlflow_client = MlflowClient()
model_versions = mlflow_client.search_model_versions(
    filter_string=f"name = '{model_name}'"
)

In [20]:
model_uri

('models:/heart_disease_model/Production',)

In [12]:
version = "Production"
model_uri = f"models:/{model_name}/{version}"
mlflow.pyfunc.load_model(model_uri)

c:\Users\ASUS\Anaconda3\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


2023/12/31 20:07:25 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.9.2, required: mlflow==2.4)
 - cloudpickle (current: 2.0.0, required: cloudpickle==2.2.1)
 - numpy (current: 1.21.5, required: numpy==1.25.0)
 - packaging (current: 21.3, required: packaging==23.0)
 - scikit-learn (current: 1.2.0, required: scikit-learn==1.2.2)
 - scipy (current: 1.9.1, required: scipy==1.10.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2023/12/31 20:07:25 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.11`, differs from the version of Python that is currently running, `Python 3.9.13`, and may be incompatible
c:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator OneHotEncoder from version 

mlflow.pyfunc.loaded_model:
  artifact_path: heart_disease_RandomForestClassifier
  flavor: mlflow.sklearn
  run_id: c181a6c4-3f0f-42e6-8e8f-6c6c6167194f

In [6]:
model_versions

[<ModelVersion: aliases=[], creation_timestamp=1702959727174, current_stage='Production', description='', last_updated_timestamp=1702959791455, name='heart_disease_model', run_id='c181a6c4-3f0f-42e6-8e8f-6c6c6167194f', run_link='', source='azureml://centralus.api.azureml.ms/mlflow/v2.0/subscriptions/fe465efa-7c36-4422-ab58-265e48560849/resourceGroups/machine-learning/providers/Microsoft.MachineLearningServices/workspaces/machine-learning-space/experiments/d8fd18d1-884d-4a8d-8f4a-8eddef71c4d2/runs/c181a6c4-3f0f-42e6-8e8f-6c6c6167194f/artifacts/heart_disease_RandomForestClassifier', status='READY', status_message='', tags={}, user_id='', version='1'>]